In [87]:
#Die ganzen packages die für den versuch verwendet werden

import numpy as np #numpy ist ein package mit allen mathematischen ausdrücken/funktionen/operationen/etc. 
from uncertainties import ufloat #Uncertainties ist für das automatische berrechnen von unsicherheiten
from uncertainties.umath import * #Importieren von rechnen mit uncertainties (addition/multiplikation/etc.)
from uncertainties import unumpy as unp #kann arrays von unsicherheiten verstehen (also eine liste, statt jede zahl einzeln einzutippen)
from IPython.display import Latex, HTML, Math, display # Um ergebnisse schöner anzuzeigen. display() "ersetzt" quasi print(), und lässt latex fromatierung zu

#funktion für mittelwert mit fehler defnieren (um später ein uarray mit unsicherheiten direkt zu einem mittelwert zu machen)
def mean(values):
    nom = unp.nominal_values(values)
    std_mean = np.std(nom, ddof=1) / np.sqrt(len(nom))
    return ufloat(np.mean(nom), std_mean)


unsicherheit = 1 #mm?   #Ich nehme mal an das die gemessene unsicherheit immer gleich bleibt, also brauchen wir die nur einmal

# Geometrische Optik

## Bestimmung der Brechkraft einer Konvexlinse

### Bei unveränderter Gegenstandsweite

Durch mehrmalige Messung wird bei fixierter Gegenstandsweite die Bildweite bestimmt. Mithilfe der Linsengleichung kann dann aus Bild- und Gegenstandsweite die Brennweite und somit auch die Brechkraft in Dioptrien errechnet werden. 
    
\begin{gather}
\frac{1}{f}=\frac{1}{b}+\frac{1}{g}
\end{gather}



$$D = (6.6 \pm 0.5) \frac{1}{m}$$

### Mithilfe des Besselverfahrens

Dieses Verfahren beruht auf der Veränderung der Distanz e zwischen Schirm und Linse, wobei e größer als 4f sein muss. Daraus ergibt sich dann die neue Gleichung für die Brennweite: 

\begin{gather}
f = \frac{1}{4}(e-\frac{d^2}{e})
\end{gather}

Mit 5 verschiedenen Werten für e ergibt sich dann für die Brechkraft D: 

$$D = (6.5 \pm 0.5) \frac{1}{m}$$

Auch hier überwiegt die Ableseunsicherheit im Vergleich zu statistischen und wird deshalb als Gesamtunsicherheit angenommen.

## Bestimmung der Brechkraft einer Konkavlinse 

Weil eine Konkavlinse ein virtuelles und kein reelles Bild erzeugt, wird bei der Messung eine Konvexlinse dazugenommen. Daraus ergibt sich die neue Gegenstandsweite

\begin{gather}
g_2 = -(b_1-d)
\end{gather}

Danach wird die Bildweite $b_2$  der Konkavlinse gemessen. d beschreibt den Abstand der beiden Linsen und $b_1$ ist die Bildweite der Konvexlinse.

Daraus ergibt sich dann die Brechkraft

$$D = (-2.3 \pm 0.5) \frac{1}{m}$$


## Anmerkung zu den Unsicherheiten

Die von uns verwendeten Unsicherheiten sind höher als jene, die statistisch errechnet wurden. Das liegt daran, dass die Einstellung der optimalen Schärfe ebenfalls einen relativ großen Fehler mit sich zieht. Wir haben sie also als  $$ \pm 0.5\frac{1}{m}$$ angenommen. In den Messwerttabellen finden sich die exakt errechneten Unsicherheiten.






In [88]:
####################################################################################
#Punkt a,

#werte liste für die gegenstands/bildweite (entfernung von linse) g/b:
#üssen nur 5 min. sein

g_1_werte = 220   #mm #gegenstandsweite bleibt konstant
b_1_werte = np.array([1138, 1131, 1134, 1136, 1134])   #mm

#unsicherheitsabelle mit einzelnen werten erstellen:

g = ufloat(g_1_werte, unsicherheit)
b = ufloat(1630, 1) - unp.uarray(b_1_werte, unsicherheit) #position von schirm von der positionder linse abziehen um die länge b zu bekommen

#formel für brennweite f:

f = 1/(1/b+1/g)       #mm

#f ist ein wert mit unsicherheit f = f_nom +- f_std
f_nom = unp.nominal_values(f)       #nominal-werte von f (also nur die werte)
f_std = unp.std_devs(f)             #standard deviation von f (also nur unsicherheit)

#-----------------------------------------------------------------------------------
#ab hier nur noch um die daten schon zu darzustellen:

#Latex tabelle definieren
table = r"""$$
\begin{array}{c|c}
\textbf{} & \textbf{Brennweite} ~f~[\mathrm{mm}] \\ \hline
"""
#für jedes i-te paar (nom,std) in der numerierten liste [(1,0.1),(2,0.1),(3,0.1)...] fügen wir der vorher definierten tabelle einen neuen eintrag ein:
#(table += rf"{i-ter eintrag} & wertepaar"). Wenn keine i mehr übrig sind hört der loop mit table += r"\end{array}" auf.
for i, (nom, std) in enumerate(zip(f_nom, f_std)):
    table += rf"{i} & ({nom:.2f} \pm {std:.2f}) \\ "
table += rf"\\ {{Schnitt}} & ({unp.nominal_values(mean(f)):.2f} \pm {unp.std_devs(mean(f)):.2f})" #bevor der loop endet wird noch in die letzte zeile der mittelwert dazugegeben
table += r"\end{array}$$"

#Tabelle anzeigen lassen (printen)
display(Latex(table))


display(Latex(rf"$${{\frac{1}{{f}}}} = D = ({1/mean(f)*1000:.2f}~\mathrm){{\frac{1}{{m}}}}$$"))
display(Latex(rf"$$D = ({1/(unp.nominal_values(mean(f))/1000):.2f}\pm 0.5~\mathrm){{\frac{1}{{m}}}}$$"))
#-----------------------------------------------------------------------------------

####################################################################################

####################################################################################
#Punkt b, Besselverfahren

#werte liste für e (gegenstand-schirm abstand) und d = d_2 - d_1 (verschiebung/symmetrische linsenposition):


e =   ufloat(1850, 1) - unp.uarray([1138, 1080, 1050, 1000, 980], unsicherheit) #mm  #position von objekt minus position von schirm ist e
d_1 =  unp.uarray([1360, 1290, 1261, 1202, 1183], unsicherheit) #mm  #erste linsen position
d_2 = unp.uarray([1630, 1645, 1647, 1655, 1658], unsicherheit) #mm #zweite linsen postion

d = d_2-d_1 #linsen postionen voneinander abziehen um total zu bekommen.

#formel für brennweite f:

f_1 = (1/4)*(e-((d**2)/e))       #mm

f_nom_1 = unp.nominal_values(f_1)
f_std_1 = unp.std_devs(f_1)

display(Latex(r"$$\mathrm{\textbf{Besselverfahren}}$$"))
table = r"""$$
\begin{array}{c|c}
\textbf{} & \textbf{Brennweite} ~f~[\mathrm{mm?}] \\ \hline
"""
for i, (nom_1, std_1) in enumerate(zip(f_nom_1, f_std_1)):
    table += rf"{i} & ({nom_1:.2f} \pm {std_1:.2f}) \\ "
table += rf"\\ {{Schnitt}} & ({unp.nominal_values(mean(f_1)):.2f} \pm {unp.std_devs(mean(f_1)):.2f})"
table += r"\end{array}$$"

display(Latex(table))
#Zeigt den berrechneten wert 1/mittelwert an und konvertiert von mm in m (1/1000)
display(Latex(rf"$${{\frac{1}{{f}}}} = D = ({1/(mean(f_1)/1000):.2f}~\mathrm){{\frac{1}{{m}}}}$$"))
#!!!!!!!!!!!!!UNSICHERHEIT IST VON UNS EINGETRAGEN WORDEN
display(Latex(rf"$$D = ({1/(unp.nominal_values(mean(f_1))/1000):.2f}\pm 0.5~\mathrm){{\frac{1}{{m}}}}$$"))
####################################################################################


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [89]:
#Konkavlinse Brennweite messen

#Konvexlinse daten
g_1_werte = np.array([405 , 395])   #mm?
b_1_werte = np.array([244 , 253])   #mm?

#Konkavlinse daten
d_werte = np.array([64 , 50]) #Abstand der beiden linsen
b_2_werte = np.array([286 , 351]) #abstand zum schirm

d = unp.uarray(d_werte, unsicherheit)
b_2 = unp.uarray(b_2_werte, unsicherheit)

g_1 = unp.uarray(g_1_werte, unsicherheit) 
b_1 = unp.uarray(b_1_werte, unsicherheit)

g_2 = -(b_1-d)

f = 1/(1/b_2+1/g_2)       #mm?

f_nom = unp.nominal_values(f)
f_std = unp.std_devs(f)

#-----------------------------------------------------------------------------------
table = r"""$$
\begin{array}{c|c}
\textbf{} & \textbf{Brennweite} ~f~[\mathrm{mm?}] \\ \hline
"""
for i, (nom, std) in enumerate(zip(f_nom, f_std)):
    table += rf"{i} & ({nom:.2f} \pm {std:.2f}) \\ "
table += rf"\\ {{Schnitt}} & ({unp.nominal_values(mean(f)):.2f} \pm {unp.std_devs(mean(f)):.2f})"
table += r"\end{array}$$"

display(Latex(table))
#Die tausend ist um von 1/mm in 1/m umzuwandeln
display(Latex(rf"$${{\frac{1}{{f}}}} = D = ({1/(mean(f)/1000):.2f}~\mathrm){{\frac{1}{{m}}}}$$"))
#!!!!!!!!!!!!!!!von mir gerunderter wert!!!!!!!!!!!!!!!
display(Latex(rf"$$D = ({1/(unp.nominal_values(mean(f))/1000):.2f}\pm 0.5 ~\mathrm){{\frac{1}{{m}}}}$$"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

# Linsenfehler

## Sphärische Aberration

Aus der Krümmung einer Linse ergeben sich für achsennahe und achsenferne Parallelstrahlen unterschiedliche Einfallswinkel auf eine sphärische Linse. Durch getrennte Beobachtung der achsennahen und -fernen Strahlen ergeben sich dann unterschiedliche Bildweiten. 

In [90]:
#Sphärische abberration

b_n = ufloat(1500, 1) - unp.uarray([1228, 1226, 1219, 1221, 1223], unsicherheit) #mm?        #Achsennahe Blende
b_f = ufloat(1500, 1) - unp.uarray([1261, 1257, 1260, 1259, 1259], unsicherheit) #mm?        #Achsenferne Blende

table = r"""$$
\begin{array}{cc}
\\ \textbf{Ferne Bildweite} ~b_f~[\mathrm{mm}] & \textbf{Nahe Bildweite} ~b_n~[\mathrm{mm}]\\ \hline \\
"""
for i, (bn, bf) in enumerate(zip(b_f, b_n), start=1):
    table += rf"({unp.nominal_values(bn):.2f} \pm {unp.std_devs(bn):.2f}) & "
    table += rf"({unp.nominal_values(bf):.2f} \pm {unp.std_devs(bf):.2f}) \\ "

#hier werden die mittelwerte in der tabelle angezeigt 
table += r"\\ \hline \\"
table += rf"\bar{{b_f}}=({unp.nominal_values(mean(b_f)):.2f} \pm {unp.std_devs(mean(b_f)):.2f}) mm & "
table += rf"\bar{{b_f}}=({unp.nominal_values(mean(b_n)):.2f} \pm {unp.std_devs(mean(b_n)):.2f}) mm\\ "

#hier werden die letzendlichen werte in der tabelle angezeigt 
# 
# !!!!!!!!!!(die unsicherheit wurden auf 5mm gerundet)!!!!!!!!!!
table += rf"\\ b_f = ({unp.nominal_values(mean(b_f)):.0f} \pm  5) mm &"
table += rf" b_n = ({unp.nominal_values(mean(b_n)):.0f} \pm  5) mm\\"
table += r"\end{array}$$"

display(Latex(table))



<IPython.core.display.Latex object>

## Chromatische Aberration

Da durch verschiedene Frequenzen die jeweiligen Lichtstrahlen unterschiedlich stark gebrochen werden, haben verschieden Farben verschiedene Brennpunkte. Um verschieden Farben mit verschiedenen Frequenzen zu messen, werden Farbfilter verwendet. Intuitiv sollte blaues Licht am stärksten gebrochen werden, da die Bedingungen in diesem Experiment allerdings nicht optimal waren, sind die Messergebnisse genau anders herum, wenn auch nur mit minimalen Unterschieden von Farbe zu Farbe. Die Umstände haben also kein schlüssiges Ergebnis zur Folge.

In [91]:
#Chromatische Aberration


b_r = ufloat(900, 1) - unp.uarray([708 , 708, 712], unsicherheit) #mm?
b_g = ufloat(900, 1) - unp.uarray([709 , 711, 706], unsicherheit) #mm?
b_b = ufloat(900, 1) - unp.uarray([708 , 708 , 707], unsicherheit) #mm?


table = r"""
$$
\begin{array}{ccc}
\\ \textbf{Rote Bildweite}~b_r~[\mathrm{mm}] & \textbf{Grüne Bildweite}~b_g~[\mathrm{mm}] & \textbf{Blaue Bildweite}~b_b~[\mathrm{mm}] \\ \\ \hline \\
"""
for i, (br, bg, bb) in enumerate(zip(b_r, b_g, b_b), start=1):
    table += rf"({unp.nominal_values(br):.2f} \pm {unp.std_devs(br):.2f}) & "
    table += rf"({unp.nominal_values(bg):.2f} \pm {unp.std_devs(bg):.2f}) & "
    table += rf"({unp.nominal_values(bb):.2f} \pm {unp.std_devs(bb):.2f}) \\"

#hier werden die mittelwerte in der tabelle angezeigt
table += r"\\ \hline \\"
table += rf"({unp.nominal_values(mean(b_r)):.2f} \pm {unp.std_devs(mean(b_r)):.2f}) & "
table += rf"({unp.nominal_values(mean(b_g)):.2f} \pm {unp.std_devs(mean(b_g)):.2f}) & "
table += rf"({unp.nominal_values(mean(b_b)):.2f} \pm {unp.std_devs(mean(b_b)):.2f}) \\ "

#hier werden die letzendlichen werte in der tabelle angezeigt 
# 
# !!!!!!!!!!(die unsicherheit wurden auf 5mm gerundet)!!!!!!!!!!
table += rf"\\ b_r = ({unp.nominal_values(mean(b_r)):.0f} \pm 5) mm & "
table += rf" b_g =({unp.nominal_values(mean(b_g)):.0f} \pm 5)mm & "
table += rf" b_b =({unp.nominal_values(mean(b_b)):.0f} \pm 5)mm \\ "
table += r"\end{array}$$"

display(Latex(table))

<IPython.core.display.Latex object>

## Anmerkung zu den Unsicherheiten

Die Unsicherheiten für die Versuche zur Aberration wurden aus Teil 1 zur Geometrischen Optik übernommen. 